<a href="https://colab.research.google.com/github/JoseFerrer/Deep_Learning_fot_Teaching/blob/main/Modelo_de_Hodgkin_Huxley_ant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modelo de Hodgkin–Huxley**
Este `notebook`está basado en el trabajo de [@amasky](https://github.com/amasky/hodgkin-huxley-model)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

En el paper [A quantitative description of membrane current and its application to conduction and excitation in nerve](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1392413/) de A. L. Hodgkin and A. F. Huxley (1952), se presenta un modelo que explica los mecanismos iónicos subyacenes al inicio y propagación de los potenciales de acción en axón del calamar gigante. Este trabajo recibió el premio nobel de fisiología en 1963.

### **Axón del calamar gigante**

![](https://upload.wikimedia.org/wikipedia/commons/1/19/Giant_Axon_of_Squid_%2814356033761%29.jpg)

### **Modelo de Hodgkin-Huxley**

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/98/Hodgkin-Huxley.svg/512px-Hodgkin-Huxley.svg.png)

### **Definir las constantes y time steps**

In [ ]:
dt = 0.01      # paso de tiempo
T = 5000       # cantidad de pasos simulados
C_m = 1.       # capacitancia de membrana (microF/cm**2)
G_k = 40.      # conductancia máxima potasio (mS/cm**2)
G_l = 0.24     # máxima fuga de conductancia (mS/cm**2)
G_na = 120.    # máxima conductancia de sodio (mS/cm**2)
V_k = -12.0    # potencial reverso de potasio (mV)
V_l = 10.613   # fuga de potencial (mV)
V_na = 115.    # potencial reverso de sodio (mV)

##**Las 4 ecuaciones del modelo de  Hodgkin-Huxley**


* $C_M\displaystyle \frac{dV}{dt}=-g_{Na}(V-V_{Na})-g_k(V-V_K)-g_l(V-V_l)+I$ ($V$: Potential de membrana)  
* $\displaystyle \frac{dh}{dt}=\alpha_h{(V)}(1-h)-\beta_h(V)h$ ($h$:  Inactivación de Sodio)  
* $\displaystyle \frac{dm}{dt}=\alpha_m{(V)}(1-m)-\beta_m(V)m$ ($m$: Activación de Sodio)  
* $\displaystyle \frac{dn}{dt}=\alpha_n{(V)}(1-n)-\beta_n(V)n$ ($n$:Activación de Potasio)  

In [ ]:
def dv(v, param, dt):
    h, m, n, i = param
    gna = G_na * h * (m**3)
    gk = G_k * (n**4)
    gl = G_l
    ina = gna * (v - V_na)
    ik = gk * (v - V_k)
    il = gl * (v - V_l)
    return (- ina - ik - il + i) * dt / C_m

def dh(h, param, dt):
    v = param
    alph = 0.07 * np.exp(-v / 20)
    beth = 1. / (np.exp((30 - v) / 10) + 1)
    return (alph * (1 - h) - beth * h) * dt

def dm(m, param, dt):
    v = param
    alpm = 0.1 * (25 - v) / (np.exp((25 - v) / 10) - 1)
    betm = 4. * np.exp(-v / 18)
    return (alpm * (1 - m) - betm * m) * dt

def dn(n, param, dt):
    v = param
    alpn = 0.01 * (10 - v) / (np.exp((10 - v) / 10) - 1)
    betn = 0.125 * np.exp(-v / 80)
    return (alpn * (1 - n) - betn * n) * dt

### Actualizar las variables por el método de Runge–Kutta

In [ ]:
def rk4(f, y, param, dt):
    k1 = f(y, param, dt)
    k2 = f(y + dt * k1 * 0.5, param, dt)
    k3 = f(y + dt * k2 * 0.5, param, dt)
    k4 = f(y + dt * k3, param, dt)
    return (k1 + 2*k2 + 2*k3 + k4) / 6

def update(v, h, m, n, i, dt):
    v += rk4(dv, v, (h, m, n, i), dt)
    h += rk4(dh, h, v, dt)
    m += rk4(dm, m, v, dt)
    n += rk4(dn, n, v, dt)
    return v, h, m, n

### Initializar las variables

In [ ]:
#@title **Simulación** { run: "auto" }

#@markdown  **Potencial de membrana [mV]**
#vt = 0. # potential de membrana (mV)
vt = 0 #@param {type:"slider", min:-90, max:50, step:10}
#mt = 0. # Canal de activación de sodio (sin unidades)
mt = 1 #@param {type:"slider", min:0, max:10, step:1}
#nt = 0. # Canal de activación de potasio (sin unidades)
nt = 1 #@param {type:"slider", min:0, max:10, step:1}
#ht = 0. # Canal de inactivación de sodio (sin unidades)
ht = 1 #@param {type:"slider", min:0, max:10, step:1}


# corriente total de membrana (microA/cm**2)
it = [10] * 500 + [0] * (T - 500) 

### Definir la corriente de membrana

In [ ]:
plt.figure(figsize=(15,5))
plt.xlabel('$t$ (ms)')
plt.ylabel('$I$ $\mathrm{(\mu/cm^2)}$')
plt.xlim(-1, 30)
plt.ylim(-2, 12)
xticks = [x * dt for x in range(-500, 0)] + [x * dt for x in range(T)]
plt.plot(xticks, [0]*500 + it)
plt.title('Corriente de membrana $I$')
plt.show()

### Obtener los valores en cada paso de tiempo

In [ ]:
vts, hts, mts, nts = ([] for i in range(4))
for t in range(int(100 // dt)):
    vt, ht, mt, nt = update(v=vt, h=ht, m=mt, n=nt, i=0, dt=dt)
    # deshacer los resultados de 100 (ms)
for t in range(T):
    vt, ht, mt, nt = update(v=vt, h=ht, m=mt, n=nt, i=it[t], dt=dt)
    vts.append(vt)
    hts.append(ht)
    mts.append(mt)
    nts.append(nt)

###Potencial de membrana

In [ ]:
plt.figure(figsize=(15,5))
plt.xlabel('$t$ (ms)')
plt.ylabel('$V$ (mV)')
plt.xlim(0, 30)
plt.ylim(-40, 120) # -20, 120
xticks = [x * dt for x in range(T)]
plt.plot(xticks, vts)
plt.title('Potencial de membrana $V$')
plt.show()

###La activación de cada canal

In [ ]:
plt.figure(figsize=(15,5))
plt.xlabel('$t$ (ms)')
plt.xlim(0, 30)
plt.ylim(-0.1, 1.1)
xticks = [x * dt for x in range(T)]
plt.plot(xticks, hts)
plt.plot(xticks, mts)
plt.plot(xticks, nts)
plt.legend(['h (Inactivación de sodio)', 
            'm (Activación de sodio)',
            'n (Activación de potasio)'],
           loc=1)
plt.title('Activación de canal iónico')
plt.show()

## Potenciales múltiples

In [ ]:
vt = mt = nt = ht = 0.
it = [10] * T
vts, hts, mts, nts = ([] for i in range(4))
for t in range(int(100 // dt)):
    vt, ht, mt, nt = update(v=vt, h=ht, m=mt, n=nt, i=0, dt=dt)
for t in range(T):
    vt, ht, mt, nt = update(v=vt, h=ht, m=mt, n=nt, i=it[t], dt=dt)
    vts.append(vt)
    hts.append(ht)
    mts.append(mt)
    nts.append(nt)

In [ ]:
plt.figure(figsize=(15,5))
plt.xlabel('$t$ (ms)')
plt.ylabel('$I$ $\mathrm{(\mu/cm^2)}$')
plt.xlim(-5, 50)
plt.ylim(-2, 12)
xticks = [x * dt for x in range(-500, 0)] + [x * dt for x in range(T)]
plt.plot(xticks, [0]*500 + it)
plt.title('Corriente de membrana $I$')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.xlabel('$t$ (ms)')
plt.ylabel('$V$ (mV)')
plt.xlim(0, 50)
plt.ylim(-20, 120)
xticks = [x * dt for x in range(T)]
plt.plot(xticks, vts)
plt.title('Potencial de membrana $V$')
plt.show()


## Periodo refractario o hiperpolarización

In [ ]:
vt = mt = nt = ht = 0.
it = ([10] * 200 + [0] * 800) * (T // 1000)
vts, hts, mts, nts = ([] for i in range(4))
for t in range(int(100 / dt)):
    vt, ht, mt, nt = update(v=vt, h=ht, m=mt, n=nt, i=0, dt=dt)
for t in range(T):
    vt, ht, mt, nt = update(v=vt, h=ht, m=mt, n=nt, i=it[t], dt=dt)
    vts.append(vt)
    hts.append(ht)
    mts.append(mt)
    nts.append(nt)

In [ ]:
plt.figure(figsize=(15,5))
plt.xlabel('$t$ (ms)')
plt.ylabel('$I$ $\mathrm{(\mu/cm^2)}$')
plt.xlim(-5, 30)
plt.ylim(-2, 12)
xticks = [x * dt for x in range(-500, 0)] + [x * dt for x in range(T)]
plt.plot(xticks, [0]*500 + it)
plt.title('Corriente de membrana  $I$')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.xlabel('$t$ (ms)')
plt.ylabel('$V$ (mV)')
plt.xlim(0, 30)
plt.ylim(-20, 120)
xticks = [x * dt for x in range(T)]
plt.plot(xticks, vts)
plt.title('Potencial de membrana $V$')
plt.show()

### **Ejercicio Práctico**



*   Controle el ancho del tren de pulsos correspondiente a la simulación del Periodo refractario o hiperpolarización, ¿qué ocurre? 




In [ ]:
### Desarrolle su código aquí

!wget -O radio.jpg https://analesranm.es/wp-content/uploads/2020/numero_137_02/images/rev05_fig04.jpg
img=plt.imread('radio.jpg')
plt.imshow(img)

